# Getting Started with najaeda

Welcome to **najaeda**! This guide will help you quickly install and start using the `najaeda` Python package.

## Installation

To install `najaeda`, simply use pip:

In [3]:
pip install najaeda

  Using cached najaeda-0.1.21-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.5 kB)
Using cached najaeda-0.1.21-cp313-cp313-macosx_11_0_arm64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


## Import najaeda
Now we will import najaeda needed package:

In [8]:
from najaeda import netlist

## Load Liberty file
Let's load a liberty file for loading primitives. 

In [9]:
liberty_files = [
    '../benchmarks/liberty/fakeram45_64x32.lib',
    '../benchmarks/liberty/fakeram45_1024x32.lib',
    '../benchmarks/liberty/NangateOpenCellLibrary_typical.lib']
netlist.load_liberty(liberty_files)

## Load a verilog design
You can now load a gate level netlist using the loaded library.

In [13]:
top = netlist.load_verilog(['../benchmarks/verilog/tinyrocket.v'])

## Explore the design

In [16]:
print(top)
for term in top.get_terms():
    print(term)
for instance in top.get_child_instances():
    print(instance)

RocketTile
clock[None:None]
reset[None:None]
auto_intsink_in_sync_0[None:None]
auto_int_in_xing_in_1_sync_0[None:None]
auto_int_in_xing_in_0_sync_0[None:None]
auto_int_in_xing_in_0_sync_1[None:None]
auto_tl_slave_xing_in_a_ready[None:None]
auto_tl_slave_xing_in_a_valid[None:None]
auto_tl_slave_xing_in_a_bits_opcode[2:0]
auto_tl_slave_xing_in_a_bits_param[2:0]
auto_tl_slave_xing_in_a_bits_size[2:0]
auto_tl_slave_xing_in_a_bits_source[4:0]
auto_tl_slave_xing_in_a_bits_address[31:0]
auto_tl_slave_xing_in_a_bits_mask[3:0]
auto_tl_slave_xing_in_a_bits_data[31:0]
auto_tl_slave_xing_in_d_ready[None:None]
auto_tl_slave_xing_in_d_valid[None:None]
auto_tl_slave_xing_in_d_bits_opcode[2:0]
auto_tl_slave_xing_in_d_bits_param[1:0]
auto_tl_slave_xing_in_d_bits_size[2:0]
auto_tl_slave_xing_in_d_bits_source[4:0]
auto_tl_slave_xing_in_d_bits_sink[None:None]
auto_tl_slave_xing_in_d_bits_denied[None:None]
auto_tl_slave_xing_in_d_bits_data[31:0]
auto_tl_slave_xing_in_d_bits_corrupt[None:None]
auto_tl_maste